# Analysis of extrinsic results

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pickle
import json
import pandas as pd
import seaborn as sns
import plotly.express as px
import numpy as np
from scipy import stats
from pysal.explore import esda
from pysal.lib import weights
from splot.esda import lisa_cluster

%run ../settings/yaml_variables.py
%run ../settings/df_styler.py
%run ../settings/plotting.py

<string>:49: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
/Users/anev/Library/CloudStorage/Dropbox/ITU/repositories/bikedna_dk_analysis/settings/plotting.py:49: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap_name, n)


In [2]:
# Import summary dataframe
summarize_ex_results_df = pd.read_csv(f"../results/COMPARE/{study_area}/data/extrinsic_summary_results.csv",index_col=0)

summarize_ex_results_df.style.pipe(format_extrinsic_style)

,OSM,GeoDanmark
Total infrastructure length (km),"20,619","8,676"
Protected bicycle infrastructure density (m/km2),383,92
Unprotected bicycle infrastructure density (m/km2),61,95
Mixed protection bicycle infrastructure density (m/km2),1,0
Bicycle infrastructure density (m/km2),445,187
Nodes,"90,419","51,224"
Dangling nodes,"46,344","11,218"
Nodes per km2,2,1
Dangling nodes per km2,1,0
Overshoots,92,99


In [3]:
# Import summary dataframe
summarize_fm_results_df = pd.read_csv(
    f"../results/COMPARE/{study_area}/data/feature_matching_summary_stats.csv",
    index_col=0,
)

summarize_fm_results_df.style.pipe(format_matched_style)


,OSM,GeoDanmark
Count of matched edges,"17,014","32,055"
Percent matched edges,19%,63%
Length of matched edges (km),"3,515","5,600"
Percent of matched network length,23%,65%
Local min of % matched edges,1%,2%
Local max of % matched edges,100%,100%
Local average of % matched edges,53%,88%


In [ ]:
# TODO: Analyze SPATIAL PATTERN in differences in density

# TODO: Analyze SPATIAL PATTERN in differences in matched features

# TODO: Find a way to compare differences in density to matched features

# TODO: Analysis of overlap between errors and differences.

# TODO: Analysis of correlation between high/low quality areas and pop density

In [4]:
make_plots = True

In [ ]:
# Create/load spatial weights

%run ../settings/create_osm_grid_spatial_weights.py

adj_list = pd.read_csv(osm_grid_spatial_weights_fp)

w = weights.W.from_adjlist(adj_list)